In [1]:
import os

In [2]:
!pwd
os.chdir('../')
!pwd

/Users/Bingumalla Likith/Desktop/MLOPS/Project-2/research
/Users/Bingumalla Likith/Desktop/MLOPS/Project-2


In [6]:
from dataclasses import dataclass
from pathlib import Path
from box import ConfigBox

@dataclass
class ModelEvaluationConfig:
    target_column: str
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    best_params_path: ConfigBox

In [7]:
from src.data_science.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.data_science.utils.common import read_yaml, create_directories, load_bin

class ConfigurationManager:
    def __init__(self,
                config_path = CONFIG_FILE_PATH,
                params_path = PARAMS_FILE_PATH,
                schema_path = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.schema = read_yaml(schema_path)

        create_directories(self.config.artifacts_root)
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        model_evaluation_config = ModelEvaluationConfig(
            self.schema.TARGET,
            **self.config
        )

        return model_evaluation_config

In [9]:
from src.data_science import logger
from sklearn.metrics import mean_squared_error
import pandas as pd

class ModelEvaluater:
    def __init__(self, config = ModelEvaluationConfig):
        self.config = config
    
    def evaluate(self):
        test_data = pd.read_csv(self.config.test_data_path)
        y_test = test_data[self.config.target_column]
        x_test = test_data.drop(columns=[self.config.target_column])

        model = load_bin(self.config.model_path)
        predictions = model.predict(x_test)
        error = mean_squared_error(y_test, predictions)

        logger.info(f"Model error on test data : {error}")